In [ ]:
import yaml
import datetime
import locale
from dotenv import load_dotenv
from crewai import Agent , Task , Crew , Process
from crewai_tools import WebsiteSearchTool , SerperDevTool


locale.setlocale (locale.LC_TIME, 'pt_BR.utf8')
load_dotenv ()
search_tool = SerperDevTool()
gpt_mini = 'gpt-4o-mini'

# Tamanho do enxame
enxame = 5

In [ ]:
pesquisador_web_config = {
    "role": "Pesquisador Web",
    "backstory": """Você é um especialista em encontrar fontes online confiáveis para vários tópicos. Sua missão é fornecer uma lista de sites que os usuários podem explorar para obter mais informações sobre seus interesses específicos.""",
    "goal": "Encontrar sites relevantes para o tema {topic}, produzir uma saída YAML com o seguinte template {template}",
    "tools": [search_tool],
    "verbose": True,
    "llm": gpt_mini
}

pesquisador = Agent(config=pesquisador_web_config)

In [ ]:
encontrar_sites_config = {
    "description": """Realize uma pesquisa na web para identificar e compilar uma lista de sites relevantes para o tema fornecido. Certifique-se de que os sites são confiáveis e fornecem insights valiosos sobre o tópico. A saída final deve ser formatada como YAML. Produza uma saída yaml no seguinte template {template}""",
    "expected_output": """Uma YAML de 5 a 10 sites relacionados ao tema {topic}. Muito importante, retire coisas como '''yaml''' e ''' da saída""",
    "agent": pesquisador,
}

encontrar_sites = Task(config=encontrar_sites_config)

In [ ]:
crew_inicial = Crew(
    agents = [pesquisador],
    tasks = [encontrar_sites],
    processes = Process.sequential
)

In [ ]:
data_atual = datetime.datetime.now()
data_hoje = data_atual.strftime('%A, %d de %B de %Y')
template = """
<template>
site:
    - link: "url da notícia"
</template>
"""

In [ ]:
topic = " Preço de ó culos de sol masculinos em sites do Brasil  atualizado para " + data_hoje
input = {'topic': topic,
         'template': template,
         'sites': enxame}

In [ ]:
output = crew_inicial.kickoff( inputs = input )
yaml_string = output.raw.strip().strip("```yaml").strip("```")
data_dict = yaml.safe_load( yaml_string )
sites_list = data_dict.get('site', [])

In [ ]:
# ------------------------------
# Segunda Crew : O enxame de agentes
# ------------------------------
agentes = []
tarefas = []

for index, site in enumerate(sites_list):
    try:
        busca = WebsiteSearchTool(website=site['link'])
        
        agente = Agent(
            role=f"Pesquisador Web {index}",
            goal=f"Buscar informações sobe {topic} no '{site['link']}'",
            backstory=f"""Especializado em recuperar informações detalhadas de sites específicos, você se concentrará em explorar o conteúdo disponível no {site['link']} relacionado a {topic}.""",
            tools=[busca],
            verbose=True,
            memory=True,
            allow_delegation=False,
            llm=gpt_mini
        )
        tarefa = Task (
            description=f"""Use o site especificado {site['link']} para encontrar informações detalhadas sobre {topic}. É muito importante que o link de onde foi encontrado a informação esteja junto da inofrmação.""",
            expected_output=f"""Retornar informações sobre {topic} a partir do site {site}.""",
            agent=agente,
            output_file=f"./partial/{index}.md"
        )
        agentes.append(agente)
        tarefas.append(tarefa)
    except Exception as e:
        print(f"Erro ao criar agente {index}: {e}")

In [ ]:
agente_agregador_config = {
    "role":"Agregador",
    "goal":"Consolidar informações de todos os agentes em um documento Markdown",
    "backstory":f"""Você é um especialista em sintetizar informações de várias fontes em um formato coeso. Sua tarefa é compilar os insights coletados e apresent-a-los em um documento Markdown.""",
    "verbose":True
}

agente_agregador = Agent(config=agente_agregador_config)

tarefa_agregador_config = {
    "description":f"""Consolide os resultados dos agentes dinâmicos em um documento Markdown. Certifique-se de incluir todas as informações relevantes coletadas.""",
    "expected_output":f"""Um documento Markdown que resumo as informações coletadas de todos os sites.""",
    "agent":agente_agregador,
    "output_file":"resultado.md"
}

tarefa_agregador = Task(config=tarefa_agregador_config)

agentes.append(agente_agregador)
tarefas.append(tarefa_agregador)

In [ ]:
crew_assincrona = Crew (
    agents=agentes,
    tasks=tarefas,
    processes=Process.sequential
)

result = crew_assincrona.kickoff(inputs={})